table of contents
1. 基本
1. 配列

Juliaの特徴
* コンパイル言語に比肩する実行速度
* シンプルな言語機能と豊富な標準ライブラリ
* 引数の型により実行される関数が決まる動的ディスパッチ
* Lispのような強力なマクロ機能

# Juliaの文法

## Hello World

改行なし

In [1]:
print("Hello World")

Hello World

改行あり

In [2]:
println("a")

a


## 変数名

Juliaの変数名はとても自由です。英字やアンダースコア(`_`)から始まる英数字/アンダースコアの他に、UTF8の多様な文字が使えます。

使えるもの:
* `xyz`
* `_foo3_`
* `π`
* `f′`

使えないもの:
* `33xyz`などの数値から始まるもの
* `for`, `function`, `end`など予約語
* `x.y`, `x:y`などの予約されている記号を使ったもの

`x`などが使えるのはもちろんですが、

In [3]:
x = 100

100

`η`のようなギリシャ文字や漢字も使えます。

In [4]:
η = 0.01

0.01

In [5]:
漢字変数 = η

0.01

ここまで見てきたように、変数は特別に宣言せずとも初期化と同時に使用できます。

変数には、その影響するソースコードの範囲である**スコープ**という概念があります。
`function`や`for`などで始まり、`end`で終わるほとんどのブロックは新たな変数のスコープを作ります。`for ... end`がスコープを作るのはPythonなどと動作が異なりますので注意が必要です。

以下の例では、変数`xx`は`for ... end`の内側のみのスコープを持つため、その外で`xx`にアクセスすると変数未定義の例外が発生しています。

In [6]:
for i in 1:10
    xx = i
end
xx

LoadError: UndefVarError: xx not defined

例外的にスコープを作らないのは`if ... end`と`begin ... end`です。すなわち、`if ... end`や`begin ... end`の内側で定義した変数にはその外側でもアクセスできます。

In [7]:
if true
    yy = 10
end
yy

10

## 型

### 数値型

Juliaの値は型を持ちます。Juliaでは動的に型がつき、様々な機能と密接に関わってきます。

整数型は符号有無と8bit, 16bit, 32bit, 64bit, 128bitの組み合わせの10種類、それに加えて`Bool`型と`BigInt`型で合計12種類あり、それぞれ符号付き64bitは`Int64`や符号なし(unsigned)32bitは`UInt32`など一貫性のある型名がつけられています。

浮動小数点数の型も16bit, 32bit, 64bitと`BigFloat`型で合計4種類があります。型名は`Float64`などのように精度の数値が最後についています。

`BigInt`型と`BigFloat`型はそれぞれ任意精度の整数と浮動小数点数です。

In [8]:
Int64

Int64

In [9]:
Float64

Float64

他には複素数の`Complex{T}`型があります。
`T`というのは**型パラメータ(type parameter)**で、実部と虚部の数値の型を指定します。ちょうどC++のテンプレートやHaskellの型変数(type variable)のようなものです。

In [10]:
Complex{Float64}

ComplexF64 (alias for Complex{Float64})

特殊な例として、円周率のような定数は`Irrational`型として定義されています。

In [11]:
π

π = 3.1415926535897...

In [12]:
isa(π, Irrational)  # isa(x, typ)は値xが型typの値であるかどうかを返す関数

true

科学計算のために作られているJuliaは、このように豊富な数値の型を持つ点が魅力のひとつです。

### リテラル

大抵の場合、何らかの値を作るリテラルは他の言語と同じです。

* 数値
    * 整数 `Int`型: `1`, `42`, `-4`
    * 浮動小数点数 `Float64`型: `3.14`, `-2.1`, `6.0221413e+23`
    * 複素数 `Complex{T}`型: `3 + 2im`, `1.1 - 9.1im`
    * 有理数 `Rational{T}`型: `3 // 2`, `355 // 113`
* 文字(列)
    * 文字 `Char`型: `'a'`, `'樹'`
    * 文字列 `String`型: `"deadbeef"`, `"漢字"`, `"""Triple Quote String"""`
* その他
    * 真偽値 `Bool`型: `true`, `false`
    * シングルトン `Nothing`型: `nothing`
    
実際のソースコードでは`Int`型を目にすることが多いですが、これは環境によって`Int32`または`Int64`のエイリアスになっています。

型は以下のように`typeof`で確認できます。

In [13]:
typeof(42)

Int64

In [14]:
typeof(42.0)

Float64

In [15]:
typeof(3 // 2)

Rational{Int64}

In [16]:
typeof(2 + 3im)

Complex{Int64}

In [17]:
typeof('A')

Char

In [18]:
typeof('漢')

Char

In [19]:
typeof("deadbeef")

String

In [20]:
typeof("漢字")

String

In [21]:
typeof(true)

Bool

In [22]:
typeof(nothing)

Nothing

また，ある値がある型であるかは`isa`関数で確認できます。

In [23]:
isa(1, Int)

true

In [24]:
1 isa Int  # 中置もできる

true

## 型システム

Juliaにおいて、型は非常に重要な意味を持ちます。
PythonやRubyのようにJuliaは動的型付け言語ですが、オブジェクト指向のプログラミング言語での型の使い方とはかなり異なっています。
ここでは、Juliaの型システムを理解し、その後に出てくる型の定義や動的ディスパッチの前提知識を固めましょう。

### 型の階層構造

Juliaの値はひとつの **具体的な** 型を持ちます。一部例外を除いて、型が自動的に別の型にキャストされることはありません
(一部例外とは、`1 + 1.5`などの数値計算とコンストラクタです <http://julia.readthedocs.org/en/latest/manual/conversion-and-promotion/>)。

ここで、 **具体的な (concrete)** とわざわざ述べたのは、Juliaにはこれと対極をなす **抽象的な (abstract)** 型があるためです。
適切な訳語がない、ここでは原文通り具体的な型を「具体型」、抽象的な型を「抽象型」と表記します。

最も大きな違いは、**具体型はインスタンス化可能**な一方で**抽象型はインスタンス化ができない**点です。
したがって、任意の`x`に対して、`typeof(x)`の結果は必ず具体型になります。
抽象型は具体型や他の抽象型のsupertypeとして機能し、型間のsubtype/supertypeの関係性は木構造のグラフをとります。
さらに、具体型は他の型のsupertypeにはなれませんので、必然的にグラフの葉が具体型に、内部ノードが抽象型となります。
このグラフの根にあるのが`Any`という抽象型です。
Juliaでユーザーが型を定義するとデフォルトでこの`Any`型のsubtypeになります。

具体例で確認しましょう。`Int64`と`Int32`は共に具体型で、`Integer`という抽象型のsubtypeになっています。
具体型は`isconcretetype`、subtype/supertypeの関係は`<:`という関数で確認できます。

In [28]:
isconcretetype(Int64)

true

In [29]:
isconcretetype(Int32)

true

In [30]:
isconcretetype(Integer)

false

In [31]:
Int64 <: Integer

true

In [32]:
Int32 <: Integer

true

他には、`Bool`型なども`Integer`型のsubtypeです。

In [33]:
Bool <: Integer

true

全ての型はsupertypeを一つだけ持ち、抽象型は複数のsubtypeを持つことができます。
それぞれ`supertype`と`subtypes`という関数があるので、少しみてみましょう。

In [34]:
supertype(Bool)

Integer

In [36]:
subtypes(Integer)

3-element Vector{Any}:
 Bool
 Signed
 Unsigned

In [37]:
subtypes(Signed)

6-element Vector{Any}:
 BigInt
 Int128
 Int16
 Int32
 Int64
 Int8

supertypeを辿って行くと、最終的には`Any`型にたどり着きます。
次のような`tracetype`関数で検証してみましょう。

In [38]:
function tracetype(t)
    print(t)
    while t != supertype(t)
        print(" <: ")
        t = supertype(t)
        print(t)
    end
    println()
end

tracetype (generic function with 1 method)

どこから抽象型が共通になってるかなどにも注目して眺めてみて下さい。

In [39]:
tracetype(Int64)

Int64 <: Signed <: Integer <: Real <: Number <: Any


In [40]:
tracetype(Bool)

Bool <: Integer <: Real <: Number <: Any


In [41]:
tracetype(BigInt)

BigInt <: Signed <: Integer <: Real <: Number <: Any


In [42]:
tracetype(Float64)

Float64 <: AbstractFloat <: Real <: Number <: Any


In [43]:
tracetype(String)

String <: AbstractString <: Any


数値の型に関しては、以下の図のようになっています。

<img src="reports/typetree.svg">

### 型パラメータ

`[1,2,3]`は以下の様な型です。

In [44]:
typeof([1,2,3])

Vector{Int64} (alias for Array{Int64, 1})

これは、`[1,2,3]`が`Int64`型を要素とする`1`次元の配列(ベクトル)という意味になります。一般化すると、`Array{T,N}`型は`T`型を要素とする`N`次元の配列です。
このように、Juliaでは**型が別の型や値をパラメータとして持つ**ことができます。

以下の例で、型パラメータに注目して確認してください。

In [45]:
[1.0, 2.0, 3.0]

3-element Vector{Float64}:
 1.0
 2.0
 3.0

In [46]:
typeof([1.0, 2.0, 3.0])

Vector{Float64} (alias for Array{Float64, 1})

In [47]:
[1 2 3]

1×3 Matrix{Int64}:
 1  2  3

In [48]:
typeof([1 2 3])

Matrix{Int64} (alias for Array{Int64, 2})

In [49]:
[1 2 3; 4 5 6]

2×3 Matrix{Int64}:
 1  2  3
 4  5  6

In [50]:
typeof([1 2 3; 4 5 6])

Matrix{Int64} (alias for Array{Int64, 2})

### 型定義

自分で型を作るのも簡単です。
以下のように`struct`または`mutable struct`に続けて型名を書き、フィールドを定義します。

In [51]:
mutable struct Person
    name::String
    age::Int
end

struct Location
    x::Float64
    y::Float64
end

デフォルトコンストラクタがありますので、即座にインスタンス化できます。
インスタンスのフィールドへはドット(`.`)でアクセスできます。

In [52]:
person = Person("ヤマダ田中", 34)

Person("ヤマダ田中", 34)

In [53]:
person.name, person.age

("ヤマダ田中", 34)

`mutable struct`で作られた型のフィールド値は更新できます。

In [54]:
person.age += 1

35

In [55]:
person

Person("ヤマダ田中", 35)

In [56]:
loc = Location(1.0, 2.0)

Location(1.0, 2.0)

In [57]:
loc.x, loc.y

(1.0, 2.0)

`struct`で作られた型のフィールド値は更新できません。

In [58]:
loc.x += 3.0

LoadError: setfield! immutable struct of type Location cannot be changed

# 配列

### 配列 / タプル

Juliaでは1次元配列を**ベクトル(`Vector`)**、2次元配列を**行列(`Matrix`)**とよびます。

ベクトル(1次元配列)は`[x,y,...]`で表現します。

In [59]:
[1,2,3]

3-element Vector{Int64}:
 1
 2
 3

添字のアクセスはカギ括弧(`[]`)を使います。**添字は`1`から始まり**、配列の長さ分で終わります。

In [60]:
# 5要素のベクトル
xs = [1,2,3,4,5]

5-element Vector{Int64}:
 1
 2
 3
 4
 5

In [61]:
xs[1]

1

In [62]:
xs[5]

5

値の更新も標準的な構文で可能です。

In [63]:
xs[2] = 200

200

In [64]:
xs

5-element Vector{Int64}:
   1
 200
   3
   4
   5

配列を末尾からアクセスするときは`end`が使えます。

In [65]:
xs[end]

5

In [66]:
xs[end] == xs[5]

true

In [67]:
xs[end-1] == xs[4]

true

ある範囲を切り出すには`n:m`というような書き方ができます。

In [68]:
xs[2:3]

2-element Vector{Int64}:
 200
   3

In [69]:
xs[end-2:end]

3-element Vector{Int64}:
 3
 4
 5

型もちょっと見てみましょう。`[x,y,...]`で作られるのは`Array`型の値です。
以下の`{Int64,1}`の意味は後の型システムのところで説明します。

In [70]:
typeof([1,2,3])

Vector{Int64} (alias for Array{Int64, 1})

また、`[1,2,3]`は`Vector`型の値でもあります。これは`isa`関数でチェックできます。

In [71]:
isa([1,2,3], Vector)

true

行列は`[a b c; d e f]`のように書けます。

In [72]:
[1 2 3; 4 5 6]

2×3 Matrix{Int64}:
 1  2  3
 4  5  6

In [73]:
x = [1 2 3;
     4 5 6]

2×3 Matrix{Int64}:
 1  2  3
 4  5  6

添字でのアクセスも見ておきましょう。

In [74]:
x[1,2]

2

In [75]:
x[2,end]

6

In [77]:
x[1:2,end]

2-element Vector{Int64}:
 3
 6

`Vector`の時と同様に`Matrix`型であることを確認しておきます。

In [78]:
isa([1 2 3; 4 5 6], Matrix)

true

配列の要素数は`length`で取得します。

In [79]:
length([1,2,3])

3

In [80]:
length([1 2 3; 4 5 6])

6

タプル(組)は`(x,y,...)`です。

In [83]:
(1,2,3)

(1, 2, 3)

In [84]:
typeof((1,2,3))

Tuple{Int64, Int64, Int64}

タプルもベクトル同様、添字でのアクセスが出来ます

In [85]:
(1,2,3)[2]

2

In [86]:
(1,2,3)[end]

3

タプルの括弧は、曖昧性がなければ省略できます。

In [87]:
1,2,3

(1, 2, 3)

配列の大きさは`size`関数で得られますが、タプルとして返されます。

In [88]:
size([1,2,3])

(3,)

In [89]:
size([1 2 3; 4 5 6])

(2, 3)

タプルとベクトルはよく似ていますが、内部の構造や動作は大きく異なります。

まず、タプルは不変(immutable)ですが、ベクトルや配列は可変(mutable)です。
したがって、一度作ったタプルはそれ以降変更できませんが、配列では可能です。

また、タプルはメモリーの割当が起きないことがあるため、オブジェクトの生成コストが極めて小さいです。

### 範囲

Juliaには値の範囲を表す範囲型も用意されています。`start:stop`のように書くことで、`start`から`stop`まで、両端を含む範囲を表現します。

In [90]:
1:10

1:10

In [91]:
'a':'z'

'a':1:'z'

これは`for`ループを書くときや、配列や文字列から一部分を切り出す際に用いられます。

In [93]:
for i in 3:6
    println(i)
end

3
4
5
6


In [94]:
x = [1,2,3,4,5,6]

6-element Vector{Int64}:
 1
 2
 3
 4
 5
 6

In [95]:
x[3:6]

4-element Vector{Int64}:
 3
 4
 5
 6

`start:step:stop`のように書くことで、ステップ幅を指定することもできます。

In [96]:
for i in 0:10:90
    println(i)
end

0
10
20
30
40
50
60
70
80
90


ステップ幅に`-1`を指定すれば、逆順の範囲も作れます。

In [97]:
for i in 5:-1:1
    println(i)
end

5
4
3
2
1


### 辞書

Juliaでは辞書ももちろん用意されています。

In [98]:
x = Dict("foo" => 1, "bar" => 2)

Dict{String, Int64} with 2 entries:
  "bar" => 2
  "foo" => 1

In [99]:
x["foo"]

1